In [1]:
# import random
import torch
from card_engine import Card_Game

In [2]:
game = Card_Game()

for x in range(52):
    moves = game.get_legal_moves()
    i = torch.randint(len(moves), (1,))
#     print(game.turn_counter, i, moves[i])
    print(game.turn_counter, game.current_player)
    chosen_move = moves[i]
    game.play_card(chosen_move)

game.tricks_won

tensor(0) tensor(0)
tensor(1) tensor(1)
tensor(2) tensor(2)
tensor(3) tensor(3)
tensor(4) tensor(0)
tensor(5) tensor(1)
tensor(6) tensor(2)
tensor(7) tensor(3)
tensor(8) tensor(0)
tensor(9) tensor(1)
tensor(10) tensor(2)
tensor(11) tensor(3)
tensor(12) tensor(0)
tensor(13) tensor(1)
tensor(14) tensor(2)
tensor(15) tensor(3)
tensor(16) tensor(0)
tensor(17) tensor(1)
tensor(18) tensor(2)
tensor(19) tensor(3)
tensor(20) tensor(0)
tensor(21) tensor(1)
tensor(22) tensor(2)
tensor(23) tensor(3)
tensor(24) tensor(0)
tensor(25) tensor(1)
tensor(26) tensor(2)
tensor(27) tensor(3)
tensor(28) tensor(1)
tensor(29) tensor(2)
tensor(30) tensor(3)
tensor(31) tensor(0)
tensor(32) tensor(1)
tensor(33) tensor(2)
tensor(34) tensor(3)
tensor(35) tensor(0)
tensor(36) tensor(3)
tensor(37) tensor(0)
tensor(38) tensor(1)
tensor(39) tensor(2)
tensor(40) tensor(3)
tensor(41) tensor(0)
tensor(42) tensor(1)
tensor(43) tensor(2)
tensor(44) tensor(1)
tensor(45) tensor(2)
tensor(46) tensor(3)
tensor(47) tensor(0)
te

C:\Users\evarv\Documents\Erdos\dql-trick-game\utils\card_engine.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  card = torch.nn.functional.one_hot(torch.tensor(deck_index, dtype=int), self.num_cards * self.num_players).flatten()


tensor([6, 3, 2, 2])

In [3]:
game.state.nonzero()[:,1].unflatten(0, (13, 4))

tensor([[ 5,  3, 37, 11],
        [38, 33, 36, 27],
        [10,  9, 32, 12],
        [ 4, 18, 25,  0],
        [ 7, 20, 29,  6],
        [35, 26, 34, 31],
        [ 8, 47, 23, 19],
        [46, 44, 45, 50],
        [48, 41, 51,  1],
        [24, 21, 17, 15],
        [22, 13, 43, 39],
        [40, 49, 30, 14],
        [28, 16,  2, 42]])